In [257]:
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import numpy as np

Собираем все ссылки объявлений на заданном количестве страниц

In [169]:
ads = []
for page in range(20):
    driver = webdriver.Chrome()
    driver.get('https://www.cars.com/shopping/results/?list_price_max=&makes[]=&maximum_distance=20&models[]=&page={}&stock_type=all&zip=33377'.format(page))
    elements = driver.find_elements(By.CLASS_NAME, 'vehicle-card-link')
    time.sleep(2)
    for element in elements:
        ads.append(element.get_attribute('href'))
    driver.quit()

Собираем нужную нам информацию из каждого нашего объявления 

In [194]:
names = []
params = []
prices = []
description = []

for ad in ads:
    driver = webdriver.Chrome()
    driver.get(ad)
    time.sleep(3)
    names.append((driver.find_element(By.CLASS_NAME, 'listing-title')).text)
    params.append((driver.find_element(By.CLASS_NAME, 'fancy-description-list')).text)
    prices.append((driver.find_elements(By.CLASS_NAME, 'primary-price'))[1].text)
    description.append((driver.find_element(By.CLASS_NAME, 'sellers-notes')).text)
    driver.quit()

Разделим наши парметры с помощью функции

In [287]:
def params_partition(params):
    params_dict ={}
    for p in params:
        splitted_p = p.split('\n')
        if len(splitted_p) < 20:
            params_dict.setdefault('exterior_color', []).append(splitted_p[1])
            params_dict.setdefault('interior_color', []).append(splitted_p[3])
            params_dict.setdefault('drivetrain', []).append(splitted_p[5])
            params_dict.setdefault('MPG', []).append(np.nan)
            params_dict.setdefault('fuel_type', []).append(splitted_p[7])
            params_dict.setdefault('transmission', []).append(splitted_p[9])
            params_dict.setdefault('engine', []).append(splitted_p[11])
            params_dict.setdefault('VIN', []).append(splitted_p[13])
            params_dict.setdefault('stock', []).append(splitted_p[15])
            params_dict.setdefault('mileage', []).append(splitted_p[17])       
        else:
            params_dict.setdefault('exterior_color', []).append(splitted_p[1])
            params_dict.setdefault('interior_color', []).append(splitted_p[3])
            params_dict.setdefault('drivetrain', []).append(splitted_p[5])
            params_dict.setdefault('MPG', []).append(splitted_p[7])
            params_dict.setdefault('fuel_type', []).append(splitted_p[9])
            params_dict.setdefault('transmission', []).append(splitted_p[11])
            params_dict.setdefault('engine', []).append(splitted_p[13])
            params_dict.setdefault('VIN', []).append(splitted_p[15])
            params_dict.setdefault('stock', []).append(splitted_p[17])
            params_dict.setdefault('mileage', []).append(splitted_p[19])
            
    return  (params_dict) 
        
    

In [288]:
params_dict = params_partition(params)

In [289]:
params_dict['names'] = names
params_dict['description'] = description
params_dict['prices'] = prices

Создадим датасет из наших данных

In [290]:
Cars_com_data = pd.DataFrame.from_dict(params_dict, orient='index').transpose()

In [291]:
Cars_com_data.head(5)

,exterior_color,interior_color,drivetrain,MPG,fuel_type,transmission,engine,VIN,stock,mileage,names,description,prices
0,Summit White,Black,Four-wheel Drive,–,Gasoline,6-Speed Automatic,6.6L V8 16V GDI OHV,2GC4YME79P1731662,C3131,3 mi.,2023 Chevrolet Silverado 2500 Custom,2023 Chevrolet Silverado 2500HD Custom 4D Crew...,"$54,865"
1,Midnight Emerald,Saddle,All-wheel Drive,–,Hybrid,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo Hybrid,SJAAT2ZV7PC019709,23B6205,797 mi.,2023 Bentley Bentayga Hybrid Azure,Bentley Beverly Hills is proud to present this...,"$246,150"
2,Lunar Silver Metallic,Black,FWD,–,Hybrid,Automatic CVT,2.0L I4 16V GDI DOHC Hybrid,2HKRS5H50PH714500,STK714500,2 mi.,2023 Honda CR-V Hybrid Sport,This CR-V Hybrid is in transit. We are happy t...,"$34,245"
3,Carpathian Grey,Ebony,Four-wheel Drive,–,Gasoline,Automatic,2.0L I4 16V GDI DOHC Turbo,SALCT2FX3PH919614,J23014,"13,615 mi.",2023 Land Rover Discovery Sport S R-Dynamic,South Shore Protection Package includes: Stand...,"$45,995"
4,Sterling Gray Metallic,Jet Black,Four-wheel Drive,–,Diesel,10-Speed Automatic with Overdrive,3.0L I6 24V DDI DOHC Turbo Diesel,1GNSKTKT3PR367211,T230903,2 mi.,2023 Chevrolet Tahoe High Country,Sterling Gray Metallic 2023 Chevrolet Tahoe Hi...,"$82,360"


In [292]:
Cars_com_data.to_csv('Cars_com_data.csv', index=False)